## Import Libraries

In [1]:
import glob
import os
import cv2
from decimal import Decimal


## Set Paths

In [2]:
image_dir ='/home/zeela/keras-yolo3/Data/Images'
file_dir ='/home/zeela/keras-yolo3/Data/labels'

In [3]:
os.getcwd()
current_dir ='/home/zeela/keras-yolo3/Data/Images/00011.jpg' # PATH TO IMAGE DIRECTORY
im = cv2.imread(current_dir)
print(im.shape)
print(type(im.shape))

(800, 1360, 3)
<class 'tuple'>


## Conversion of x_centre, y_centre, height , width 

In [25]:
#Storing all files
files = {}
dirs = os.listdir(file_dir)

for filename in dirs:
    file = open(os.path.join(file_dir, filename), 'r')
    files[filename] = file.read() 
    file.close()

conversion = dict()

for key,value in files.items():
    lines = str.splitlines(value)
    for line in lines:
        sepline = line.split()
        label = sepline[0]
        x_centre = Decimal(sepline[1])
        y_centre = Decimal(sepline[2])
        width = Decimal(sepline[3])
        height = Decimal(sepline[4])
        ymax = int(((1600*y_centre) + (800*height)) / 2)
        ymin = int(1600*y_centre - ymax)
        xmax = int(((x_centre*2720) + (width*1360)) / 2)
        xmin = int(x_centre*2720-xmax)
        if key not in conversion.keys():
            conversion[key] = [str(xmin)+","+str(ymin)+","+str(xmax)+","+str(ymax)+","+str(label)]
        else:
            conversion[key].append(str(xmin)+","+str(ymin)+","+str(xmax)+","+str(ymax)+","+str(label))


['1010,430,1033,455,4', '474,389,500,415,0']


# Storing Bounded boxes along with Path

In [34]:
# Percentage of images to be used for the valid set
percentage_test = 10;

# Create train.txt and valid.txt
file_train = open('train.txt', 'w')  
file_test = open('valid.txt', 'w')
# Populate train.txt and valid.txt
counter = 1  
index_test = round(100 / percentage_test) 

for file in glob.iglob(os.path.join(current_dir, '*.jpg')):  
    title, ext = os.path.splitext(os.path.basename(file))
    if counter == index_test:
        counter = 1
        str1 = " "
        file_test.write(image_dir + "/" + title + '.jpg' +" "+ str1.join(conversion[title+'.txt']) + "\n")
    else:
        str1 = " "
        file_train.write(image_dir + "/" + title + '.jpg' +" "+ str1.join(conversion[title+'.txt']) + "\n")
        counter = counter + 1
file_train.close()
file_test.close()